In [2]:
import math

# Convert degrees and minutes to decimal degrees.
def dm_to_dd(degrees, minutes):
    return degrees + minutes / 60.0

# --- Input: points in degrees and minutes ---
# Point x: N 50° 04.980' E 014° 23.863'
x_lat = dm_to_dd(50, 4.980)
x_lon = dm_to_dd(14, 23.863)

# Point y: N 50° 05.250' E 014° 25.283'
y_lat = dm_to_dd(50, 5.250)
y_lon = dm_to_dd(14, 25.283)

# Given distance in meters (circle radius)
d = 4500.0

# For small areas we can use an equirectangular approximation.
# We choose point x as the origin of our local coordinate system.
# We also use the average latitude to compute the scaling for longitude.
lat0 = x_lat
lon0 = x_lon
lat_ref = math.radians((x_lat + y_lat) / 2)
R_earth = 6371000  # Earth radius in meters

def latlon_to_xy(lat, lon, lat0, lon0, lat_ref):
    """
    Converts lat/lon in decimal degrees to local x/y (in meters)
    using an equirectangular projection with (lat0, lon0) as the origin.
    """
    # differences in radians:
    dlon = math.radians(lon - lon0)
    dlat = math.radians(lat - lat0)
    x = R_earth * dlon * math.cos(lat_ref)
    y = R_earth * dlat
    return (x, y)

def xy_to_latlon(x, y, lat0, lon0, lat_ref):
    """
    Converts local x/y coordinates (in meters) back to lat/lon in decimal degrees.
    """
    dlat = y / R_earth  # in radians
    dlon = x / (R_earth * math.cos(lat_ref))  # in radians
    lat = lat0 + math.degrees(dlat)
    lon = lon0 + math.degrees(dlon)
    return (lat, lon)

# Convert the two given points to local coordinates.
# We set point x as (0,0).
x_coord = (0.0, 0.0)
y_coord = latlon_to_xy(y_lat, y_lon, lat0, lon0, lat_ref)

# Compute the distance between the centers in the local coordinate system.
dx = y_coord[0] - x_coord[0]
dy = y_coord[1] - x_coord[1]
d_centers = math.hypot(dx, dy)

if d_centers > 2 * d:
    raise ValueError("The circles do not intersect (centers are too far apart).")

# Compute the midpoint between the two centers.
mid_x = (x_coord[0] + y_coord[0]) / 2.0
mid_y = (x_coord[1] + y_coord[1]) / 2.0

# Distance from the first center to the midpoint.
a = d_centers / 2.0
# Height from the midpoint to the intersection points.
h = math.sqrt(d**2 - a**2)

# The intersection points are given by offsetting the midpoint perpendicularly to the line joining the centers.
# The perpendicular to (dx, dy) is (-dy, dx).
offset_x = -dy * (h / d_centers)
offset_y =  dx * (h / d_centers)

# Compute the two potential intersection points.
p1 = (mid_x + offset_x, mid_y + offset_y)
p2 = (mid_x - offset_x, mid_y - offset_y)

# Select the southern point (i.e. the one with the lower northing, i.e. lower y).
if p1[1] < p2[1]:
    chosen = p1
else:
    chosen = p2

# Convert the chosen local point back to latitude and longitude.
z_lat, z_lon = xy_to_latlon(chosen[0], chosen[1], lat0, lon0, lat_ref)

print("Southern intersection point z:")
print("Latitude:  {:.6f}".format(z_lat))
print("Longitude: {:.6f}".format(z_lon))


Southern intersection point z:
Latitude:  50.047198
Longitude: 14.427123


In [3]:
print("{:.6f}N".format(z_lat) + ", " +  "{:.6f}E".format(z_lon) )

50.047198N, 14.427123E
